In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests 
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
url_chicago = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'

In [3]:
def scrape_table_bs4(cname,cols):
    page=urllib.request.urlopen(url_chicago).read()
    soup=BeautifulSoup(page,'lxml')
    table=soup.find("table",class_=cname)
    header=[head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data=[[td.findAll(text=True)[0].strip() for td in tr.find_all("td")] 
          for tr in table.find_all("tr")]
    data= [row for row in data if len(row) == cols]
    
    raw_df=pd.DataFrame(data,columns=header)
    return raw_df



Chicago_neigh = scrape_table_bs4("wikitable",2)

print(Chicago_neigh.info(verbose=True))
Chicago_neigh.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 2 columns):
Neighborhood      246 non-null object
Community area    246 non-null object
dtypes: object(2)
memory usage: 4.0+ KB
None


,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [4]:
Chicago_neigh.shape

(246, 2)

In [5]:
#
latitude = []
longitude = []
for n in range(Chicago_neigh.shape[0]):
    address = [Chicago_neigh.iloc[n]['Neighborhood'], Chicago_neigh.iloc[n]['Community area'], 'Chicago', 'Illinois']
    geolocator = Nominatim(user_agent="foursquare_agent", timeout=5)
    location = geolocator.geocode(address)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [6]:
print(len(latitude))
print(len(longitude))

246
246


In [7]:
Chicago_neigh['latitude'] = latitude
Chicago_neigh['longitude'] = longitude
Chicago_neigh.head()

,Neighborhood,Community area,latitude,longitude
0,Albany Park,Albany Park,41.980269,-87.718485
1,Altgeld Gardens,Riverdale,41.654864,-87.600446
2,Andersonville,Edgewater,41.977139,-87.669273
3,Archer Heights,Archer Heights,41.807623,-87.727406
4,Armour Square,Armour Square,41.840311,-87.631996


In [8]:
Chicago_neigh.dropna(axis=0, how='any', inplace=True)
print(Chicago_neigh.shape)

(155, 4)


In [9]:
Chicago_neigh.head()

,Neighborhood,Community area,latitude,longitude
0,Albany Park,Albany Park,41.980269,-87.718485
1,Altgeld Gardens,Riverdale,41.654864,-87.600446
2,Andersonville,Edgewater,41.977139,-87.669273
3,Archer Heights,Archer Heights,41.807623,-87.727406
4,Armour Square,Armour Square,41.840311,-87.631996


In [13]:
#CLIENT_ID = 'OYWDHY2UEOKDMAH50IH0HVUBHTKY0A2B5404VC2O1KHZFJYA' # your Foursquare ID
#CLIENT_SECRET = 'SJXX4TYI4OQCD4YLOVOLR4GPBWJ4KTNAQ5DBBEFA3TCWKMMA' # your Foursquare Secret
CLIENT_ID = 'S4KPCJKZIODIUMGINZHTYKMOHV2I2C0N1PWL4BRXKPCZGWE5' 
CLIENT_SECRET = 'MTFOEKLBB2GMG1HX0BGC2COQJJOXOLCJDCLYHJPLZ0VBZSOU' 
VERSION = '20190801'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S4KPCJKZIODIUMGINZHTYKMOHV2I2C0N1PWL4BRXKPCZGWE5
CLIENT_SECRET:MTFOEKLBB2GMG1HX0BGC2COQJJOXOLCJDCLYHJPLZ0VBZSOU


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])
        """
        for v in results:
            venues_list.append(name),
            venues_list.append(lat), 
            venues_list.append(lng), 
            venues_list.append(v['venue']['name']), 
            venues_list.append(v['venue']['location']['lat']), 
            venues_list.append(v['venue']['location']['lng']),  
            venues_list.append(v['venue']['categories'][0]['name'])
            try:
                venues_list.append(v['venue']['rating'])
            except:
                venues_list.append("NaN")
        """

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue ID']
    
    return(nearby_venues)

In [15]:
Chicago_venues = getNearbyVenues(names=Chicago_neigh['Neighborhood'],
                                   latitudes=Chicago_neigh['latitude'],
                                   longitudes=Chicago_neigh['longitude'], radius=500
                                  )

Albany Park
Altgeld Gardens
Andersonville
Archer Heights
Armour Square
Ashburn
Avalon Park
Avondale
Back of the Yards
Belmont Gardens
Belmont Terrace
Beverly
Beverly Woods
Boystown
Bridgeport
Brighton Park
Bronzeville
Buena Park
Burnside
Cabrini–Green
Calumet Heights
Canaryville
Chatham
Chicago Lawn
Clarendon Park
Clearing East
Clearing West
Cragin
Dearborn Homes
Dearborn Park
Douglas Park
Dunning
East Beverly
East Garfield Park
East Pilsen
East Side
East Village
Edgebrook
Edgewater
Edgewater Beach
Edgewater Glen
Edison Park
Englewood
Fernwood
Forest Glen
Fuller Park
Gage Park
Galewood
Garfield Ridge
Gold Coast
Golden Gate
Goose Island
Grand Boulevard
Greektown
Groveland Park
Hamilton Park
Hegewisch
Hollywood Park
Homan Square
Humboldt Park
Hyde Park
Illinois Medical District
Irving Park
The Island
Jackowo
Jefferson Park
Kelvyn Park
Kennedy Park
Kensington
Kenwood
Kilbourn Park
Lake Meadows
LeClaire Courts
Lilydale
Lincoln Park
Lithuanian Plaza
Little Italy
Logan Square
Longwood Manor


In [16]:
print(Chicago_venues.shape) # size of data
Chicago_venues.head()

(3568, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
0,Albany Park,41.980269,-87.718485,Subway,41.981375,-87.718396,Sandwich Place,4d51b489747f6dcb70fdc3d4
1,Albany Park,41.980269,-87.718485,7-Eleven,41.983368,-87.714196,Convenience Store,4b748851f964a52042e22de3
2,Albany Park,41.980269,-87.718485,Café Descartes,41.981552,-87.718156,Coffee Shop,4c4082e6cc410f4794b4a961
3,Albany Park,41.980269,-87.718485,Campus Aquatics,41.978416,-87.717182,Gym Pool,4d66fc5e9792b1f7479b381f
4,Albany Park,41.980269,-87.718485,E.Leaven Food Truck,41.979290,-87.715994,Food Truck,4f1dacbee4b083823465eff6


In [17]:
Chicago_venues.groupby('Neighborhood').count() #the number of venues for each neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
Neighborhood,,,,,,,
Albany Park,8,8,8,8,8,8,8
Altgeld Gardens,2,2,2,2,2,2,2
Andersonville,85,85,85,85,85,85,85
Archer Heights,20,20,20,20,20,20,20
Armour Square,12,12,12,12,12,12,12
Ashburn,7,7,7,7,7,7,7
Avalon Park,15,15,15,15,15,15,15
Avondale,31,31,31,31,31,31,31
Back of the Yards,30,30,30,30,30,30,30


In [18]:
# the number of unique categories
print('There are {} uniques categories.'.format(len(Chicago_venues['Venue Category'].unique())))

There are 316 uniques categories.


In [19]:
# one hot encoding
Chicago_onehot = pd.get_dummies(Chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Chicago_onehot['Neighborhood'] = Chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Chicago_onehot.columns[-1]] + list(Chicago_onehot.columns[:-1])
Chicago_onehot = Chicago_onehot[fixed_columns]

Chicago_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
Chicago_onehot.shape

(3568, 316)

In [21]:
# group rows by neighborhood
Chicago_grouped = Chicago_onehot.groupby('Neighborhood').mean().reset_index()
Chicago_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Albany Park,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Altgeld Gardens,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Andersonville,0.000000,0.000000,0.00,0.011765,0.000000,0.00,0.0,0.0,0.011765,...,0.000000,0.000000,0.000000,0.0,0.000000,0.011765,0.000000,0.011765,0.000000,0.000000
3,Archer Heights,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.050000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Armour Square,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Ashburn,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Avalon Park,0.000000,0.066667,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Avondale,0.000000,0.032258,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.032258,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Back of the Yards,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Belmont Gardens,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.000000,...,0.000000,0.062500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
# create dataframe and display the top 5 venues for each neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Chicago_grouped['Neighborhood']

for ind in np.arange(Chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albany Park,Bus Station,Convenience Store,Asian Restaurant,Food Truck,Gym Pool
1,Altgeld Gardens,Park,Fast Food Restaurant,Women's Store,Factory,Eastern European Restaurant
2,Andersonville,Italian Restaurant,Coffee Shop,Pet Store,Lounge,Sandwich Place
3,Archer Heights,Mexican Restaurant,Bakery,Gas Station,Grocery Store,Pharmacy
4,Armour Square,Chinese Restaurant,Sandwich Place,Cosmetics Shop,Breakfast Spot,Grocery Store
5,Ashburn,Clothing Store,Mexican Restaurant,Locksmith,Fried Chicken Joint,Park
6,Avalon Park,Business Service,Burger Joint,Fast Food Restaurant,Food,Grocery Store
7,Avondale,Bakery,Mexican Restaurant,Bar,Donut Shop,Park
8,Back of the Yards,Mexican Restaurant,Pizza Place,Bank,Grocery Store,Clothing Store
9,Belmont Gardens,Fast Food Restaurant,Discount Store,Food,Entertainment Service,Flea Market


### Coffee Shop

In [24]:
Coffee_shop = Chicago_venues.loc[Chicago_venues['Venue Category'] == 'Coffee Shop']
Coffee_shop.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
2,Albany Park,41.980269,-87.718485,Café Descartes,41.981552,-87.718156,Coffee Shop,4c4082e6cc410f4794b4a961
24,Andersonville,41.977139,-87.669273,La Colombe Coffee Roasters,41.976095,-87.668577,Coffee Shop,546634ba498e2e08a21d1ce0
52,Andersonville,41.977139,-87.669273,Starbucks,41.978163,-87.668505,Coffee Shop,4a7efa6df964a52064f21fe3
67,Andersonville,41.977139,-87.669273,Colectivo Coffee Roasters,41.980619,-87.668247,Coffee Shop,5ba8cecb033693002c6c407f
239,Belmont Terrace,41.937585,-87.834795,Starbucks,41.934900,-87.835768,Coffee Shop,4bbe59b982a2ef3b57352bd2


In [25]:
Ratings = []

for venue_id in Coffee_shop['Venue ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        Ratings.append(result['response']['venue']['rating'])
    except:
        Ratings.append(np.nan)

Coffee_shop['Rating'] = Ratings
Coffee_shop.head()

C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Rating
2,Albany Park,41.980269,-87.718485,Café Descartes,41.981552,-87.718156,Coffee Shop,4c4082e6cc410f4794b4a961,5.6
24,Andersonville,41.977139,-87.669273,La Colombe Coffee Roasters,41.976095,-87.668577,Coffee Shop,546634ba498e2e08a21d1ce0,8.5
52,Andersonville,41.977139,-87.669273,Starbucks,41.978163,-87.668505,Coffee Shop,4a7efa6df964a52064f21fe3,7.7
67,Andersonville,41.977139,-87.669273,Colectivo Coffee Roasters,41.980619,-87.668247,Coffee Shop,5ba8cecb033693002c6c407f,8.1
239,Belmont Terrace,41.937585,-87.834795,Starbucks,41.934900,-87.835768,Coffee Shop,4bbe59b982a2ef3b57352bd2,6.6


In [30]:
print(Coffee_shop.info())
Coffee_shop.sort_values(by=['Rating'], inplace = True, ascending=False)
Coffee_shop.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121 entries, 2 to 3493
Data columns (total 9 columns):
Neighborhood              121 non-null object
Neighborhood Latitude     121 non-null float64
Neighborhood Longitude    121 non-null float64
Venue                     121 non-null object
Venue Latitude            121 non-null float64
Venue Longitude           121 non-null float64
Venue Category            121 non-null object
Venue ID                  121 non-null object
Rating                    46 non-null float64
dtypes: float64(5), object(4)
memory usage: 9.5+ KB
None


C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Rating
1684,Logan Square,41.928400,-87.706764,Sip of Hope,41.924656,-87.703974,Coffee Shop,5ac6e19d78782c6401c32429,9.0
622,Dearborn Park,41.866553,-87.628954,Stan's Donuts & Coffee,41.867516,-87.626402,Coffee Shop,569a2c27498e9defd6054348,9.0
839,Edgewater,41.983369,-87.663952,The Coffee Studio,41.984374,-87.669105,Coffee Shop,4a54e2b2f964a52062b31fe3,8.8
895,Edgewater Beach,41.983369,-87.663952,The Coffee Studio,41.984374,-87.669105,Coffee Shop,4a54e2b2f964a52062b31fe3,8.8
538,Clarendon Park,41.963275,-87.648840,Everybody's Coffee,41.965519,-87.653944,Coffee Shop,534d3962498e0eb23f7dfa5b,8.8


In [41]:
venue_id = Coffee_shop.iloc[0]['Venue ID'] # ID of the highest rating
print(venue_id)
url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result)
#result['response']['items'][0]['id']

5ac6e19d78782c6401c32429
{'meta': {'code': 200, 'requestId': '5d658434db1d81003922d249'}, 'response': {'likes': {'count': 34, 'summary': '34 Likes', 'items': [{'id': '40319144', 'firstName': 'Jessi', 'lastName': 'Lenth', 'gender': 'female', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/ZNBXLYXUDZALDOYU.jpg'}}, {'id': '31725619', 'firstName': 'Douglas', 'lastName': 'Shoback', 'gender': 'male', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/31725619_VS2lKy7v_t3jbXB3XQlKcOFzAEbKkhDdfpIsIhqHxSOYKdKCE8HGzGua7I_qD4cVFV1kXHGz2.jpg'}}, {'id': '462532833', 'firstName': 'Pavla', 'lastName': 'K', 'gender': 'none', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/blank_boy.png', 'default': True}}]}}}


In [49]:
venue_id = Coffee_shop.iloc[0]['Venue ID']
CLIENT_ID = 'JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR'
CLIENT_SECRET = 'KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX'
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d659a5f8fc77c0030a814d1'},
 'response': {'tips': {'count': 10,
   'items': [{'id': '5b990e1fba57b4002c5cf7f8',
     'createdAt': 1536757279,
     'text': 'Sip Dark Matter. Hang, do some work. Be pleased knowing your money is going to a very worthy cause. Remember, it’s ok to talk about your feelings & it’s important to let people tell you how they feel.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5b990e1fba57b4002c5cf7f8',
     'photo': {'id': '5b990e219e3b65002ca54303',
      'createdAt': 1536757281,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/169953_Sb6h3HtTQMFQ9Hn80v9JDodGdxoa_m_RJTA87CVZkNw.jpg',
      'width': 1080,
      'height': 1920,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/169953_Sb6h3HtTQMFQ9Hn80v9JDodGdxoa_m_RJTA87CVZkNw.j

In [52]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [53]:
from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Sip Dark Matter. Hang, do some work. Be pleased knowing your money is going to a very worthy cause. Remember, it’s ok to talk about your feelings & it’s important to let people tell you how they feel.",8,0,5b990e1fba57b4002c5cf7f8,ben,m,male,169953


In [54]:
user_id = '169953' # user ID with 

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [68]:
print(results['response']['user']['lists']['groups'])

[{'type': 'created', 'count': 24, 'items': [{'id': '59c0640b356b4956ad117922', 'name': 'SoCal To Do', 'description': '', 'editable': False, 'public': True, 'collaborative': False, 'url': '/illorange/list/socal-to-do', 'canonicalUrl': 'https://foursquare.com/illorange/list/socal-to-do', 'createdAt': 1505780747, 'updatedAt': 1566567983, 'followers': {'count': 0}, 'listItems': {'count': 85}}, {'id': '59c0683fee628b40bcca1dcb', 'name': 'Nashville To Do', 'description': '', 'editable': False, 'public': True, 'collaborative': False, 'url': '/illorange/list/nashville-to-do', 'canonicalUrl': 'https://foursquare.com/illorange/list/nashville-to-do', 'createdAt': 1505781823, 'updatedAt': 1566272703, 'followers': {'count': 0}, 'listItems': {'count': 25}}, {'id': '5b15fb78febf31002c9944fc', 'name': 'Charleston & Savannah To Do', 'description': '', 'editable': False, 'public': True, 'collaborative': False, 'url': '/illorange/list/charleston--savannah-to-do', 'canonicalUrl': 'https://foursquare.com/i

In [131]:
user_id = '169953' # user ID with 
url = 'https://api.foursquare.com/v2/users/{}/lists?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
#results['listItems']
results

{'meta': {'code': 200, 'requestId': '5d65b7b6a1979300376d686d'},
 'response': {'lists': {'count': 30,
   'groups': [{'type': 'yours',
     'name': 'Your Places',
     'count': 2,
     'items': [{'id': '169953/todos',
       'name': "ben's Saved Places",
       'description': '',
       'placesSummary': "The Barrel, Danny's Tavern, Whirlaway Lounge, Guthrie's Tavern",
       'type': 'todos',
       'user': {'id': '169953',
        'firstName': 'ben',
        'lastName': 'm',
        'gender': 'male',
        'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
         'suffix': '/169953_rEWAUPqC_FyCTl9d1zZ8_GS1NRFwWtMFaEsdG8zX1VYsiX83ocW-pdKW7v1-kOmeON4zkmPno.jpg'}},
       'editable': False,
       'public': True,
       'collaborative': False,
       'url': '/illorange/list/todos',
       'canonicalUrl': 'https://foursquare.com/illorange/list/todos',
       'updatedAt': 1566675168,
       'listItems': {'count': 310},
       'photos': {'count': 4,
        'groups': [{'type': 'venu

In [133]:
results['response']['lists']['groups'][0]['items'][1]

{'id': '169953/venuelikes',
 'name': 'ben’s Liked Places',
 'description': '',
 'placesSummary': "Joe's Pizza, Lucky Dorr Patio & Tap, Otto's Place, Au Pied de Cochon",
 'type': 'liked',
 'user': {'id': '169953',
  'firstName': 'ben',
  'lastName': 'm',
  'gender': 'male',
  'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
   'suffix': '/169953_rEWAUPqC_FyCTl9d1zZ8_GS1NRFwWtMFaEsdG8zX1VYsiX83ocW-pdKW7v1-kOmeON4zkmPno.jpg'}},
 'editable': False,
 'public': True,
 'collaborative': False,
 'url': '/illorange/list/venuelikes',
 'canonicalUrl': 'https://foursquare.com/illorange/list/venuelikes',
 'updatedAt': 1566530986,
 'listItems': {'count': 191},
 'photos': {'count': 4,
  'groups': [{'type': 'venue',
    'name': 'Venue photos',
    'count': 4,
    'items': [{'id': '4fe905cde4b038fdd45165a9',
      'createdAt': 1340671437,
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/S6oGDGI94kuRh8GHS6OgX_PZeHGwcvTsffUCaYdj4Yg.jpg',
      'width': 720,
      'height':

In [82]:
list_id = '169953/venuelikes'
url = 'https://api.foursquare.com/v2/lists/{}?client_id={}&client_secret={}&v={}'.format(list_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d65a2e0992951002577b079'},
 'response': {'list': {'id': '169953/venuelikes',
   'name': 'ben’s Liked Places',
   'description': '',
   'type': 'liked',
   'user': {'id': '169953',
    'firstName': 'ben',
    'lastName': 'm',
    'gender': 'male',
    'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
     'suffix': '/169953_rEWAUPqC_FyCTl9d1zZ8_GS1NRFwWtMFaEsdG8zX1VYsiX83ocW-pdKW7v1-kOmeON4zkmPno.jpg'}},
   'editable': False,
   'public': True,
   'collaborative': False,
   'url': '/illorange/list/venuelikes',
   'canonicalUrl': 'https://foursquare.com/illorange/list/venuelikes',
   'updatedAt': 1566530986,
   'visitedCount': 0,
   'venueCount': 30,
   'following': False,
   'followers': {'count': 0},
   'collaborators': {'count': 0, 'items': []},
   'sort': 'recent',
   'listItems': {'count': 191,
    'items': [{'id': 't53ed636c498ee6ce96d3849c',
      'user': {'id': '169953',
       'firstName': 'ben',
       'lastName': 'm',
       'gender

In [85]:
ben_likes = results['response']['list']['listItems']['items']

In [113]:
ben_likes_list = []
ben_likes_list.append([(
    place['venue']['id'],
    place['venue']['name'],
    #place['venue']['location']['city'],
    place['venue']['categories'][0]['name']) for place in ben_likes])
ben_likes_list

[[('3fd66200f964a520dbea1ee3', 'The Blind Tiger', 'Beer Bar'),
  ('45ebc982f964a52091431fe3', "Joe's Pizza", 'Pizza Place'),
  ('59653657018cbb46a640bd78', 'Lucky Dorr Patio & Tap', 'Beer Bar'),
  ('4bdc54552a3a0f477c54b2b6', 'Flying Saucer', 'Breakfast Spot'),
  ('40b28c80f964a52029fc1ee3', "Delilah's", 'Whisky Bar'),
  ('4bc9e445937ca593cc86a692', 'Tiztal Cafe', 'Breakfast Spot'),
  ('4df4e6b763659543662d05a7', "Otto's Place", 'American Restaurant'),
  ('4adb95ebf964a5202d2921e3', 'Le Garde-Manger', 'Seafood Restaurant'),
  ('4b84801af964a520fd3831e3', 'Joe Beef', 'Steakhouse'),
  ('4ad4c06bf964a52046f920e3', 'Au Pied de Cochon', 'French Restaurant'),
  ('49ea7b74f964a5206e661fe3',
   "Pappy & Harriet's Pioneertown Palace",
   'BBQ Joint'),
  ('4b4e9dc9f964a520b2f226e3', 'Carnevor Steakhouse Moderne', 'Steakhouse'),
  ('5c09d27867af3a002cb81bbc', 'The Swill Inn', 'Pub'),
  ('4c1f8dd3b306c928029768b7', 'Girl & the Goat', 'New American Restaurant'),
  ('4b05865cf964a520d05e22e3', 'Proo

In [115]:
df_ben_likes = pd.DataFrame([item for venue_list in ben_likes_list for item in venue_list])
df_ben_likes.columns = ['ID', 'Name', 'Category']
df_ben_likes

,ID,Name,Category
0,3fd66200f964a520dbea1ee3,The Blind Tiger,Beer Bar
1,45ebc982f964a52091431fe3,Joe's Pizza,Pizza Place
2,59653657018cbb46a640bd78,Lucky Dorr Patio & Tap,Beer Bar
3,4bdc54552a3a0f477c54b2b6,Flying Saucer,Breakfast Spot
4,40b28c80f964a52029fc1ee3,Delilah's,Whisky Bar
5,4bc9e445937ca593cc86a692,Tiztal Cafe,Breakfast Spot
6,4df4e6b763659543662d05a7,Otto's Place,American Restaurant
7,4adb95ebf964a5202d2921e3,Le Garde-Manger,Seafood Restaurant
8,4b84801af964a520fd3831e3,Joe Beef,Steakhouse
9,4ad4c06bf964a52046f920e3,Au Pied de Cochon,French Restaurant


In [123]:
df_ben_likes.Category.value_counts()

Beer Bar                   3
Pizza Place                3
Indian Restaurant          2
Steakhouse                 2
Breakfast Spot             2
Cocktail Bar               2
New American Restaurant    1
North Indian Restaurant    1
Bagel Shop                 1
French Restaurant          1
Korean Restaurant          1
Seafood Restaurant         1
Food                       1
BBQ Joint                  1
Italian Restaurant         1
Lounge                     1
Bar                        1
Pub                        1
Whisky Bar                 1
Chinese Restaurant         1
American Restaurant        1
Bakery                     1
Name: Category, dtype: int64

### Recommendation for Beer Bar

In [125]:
ben_likes_beer = Chicago_venues.loc[Chicago_venues['Venue Category'] == 'Beer Bar']
ben_likes_beer

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID
11,Andersonville,41.977139,-87.669273,Hopleaf Bar,41.975813,-87.668509,Beer Bar,412a8500f964a520770c1fe3
77,Andersonville,41.977139,-87.669273,Meeting House Tavern,41.973295,-87.667916,Beer Bar,5b16e138db3aef0024361aac
2209,Near North Side,41.900033,-87.634497,Centennial Crafted Beer & Eatery,41.895807,-87.632539,Beer Bar,58c1b0879b7eac37c66519f5
2450,O'Hare,41.977928,-87.902955,Goose Island Beer Co.,41.977056,-87.907214,Beer Bar,581e34531338633a85a462a3
3491,Wrigleyville,41.947022,-87.656477,Lucky Dorr Patio & Tap,41.948660,-87.656829,Beer Bar,59653657018cbb46a640bd78


In [127]:
Ratings = []

for venue_id in ben_likes_beer['Venue ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        Ratings.append(result['response']['venue']['rating'])
    except:
        Ratings.append(np.nan)

ben_likes_beer['Rating'] = Ratings
ben_likes_beer.sort_values(by=['Rating'], inplace = True, ascending=False)
ben_likes_beer

C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Rating
11,Andersonville,41.977139,-87.669273,Hopleaf Bar,41.975813,-87.668509,Beer Bar,412a8500f964a520770c1fe3,9.1
2209,Near North Side,41.900033,-87.634497,Centennial Crafted Beer & Eatery,41.895807,-87.632539,Beer Bar,58c1b0879b7eac37c66519f5,9.1
3491,Wrigleyville,41.947022,-87.656477,Lucky Dorr Patio & Tap,41.948660,-87.656829,Beer Bar,59653657018cbb46a640bd78,8.8
77,Andersonville,41.977139,-87.669273,Meeting House Tavern,41.973295,-87.667916,Beer Bar,5b16e138db3aef0024361aac,7.4
2450,O'Hare,41.977928,-87.902955,Goose Island Beer Co.,41.977056,-87.907214,Beer Bar,581e34531338633a85a462a3,7.4


### Recommendation for Pizza Place

In [128]:
ben_likes_pizza = Chicago_venues.loc[Chicago_venues['Venue Category'] == 'Pizza Place']

Ratings = []

for venue_id in ben_likes_pizza['Venue ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        Ratings.append(result['response']['venue']['rating'])
    except:
        Ratings.append(np.nan)

ben_likes_pizza['Rating'] = Ratings
ben_likes_pizza.sort_values(by=['Rating'], inplace = True, ascending=False)
ben_likes_pizza

C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\zouru\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Rating
1132,Goose Island,41.910048,-87.655765,Pizzeria Bebu,41.909237,-87.651089,Pizza Place,58962beb375c4a79fe9ecdb0,9.1
325,Boystown,41.943883,-87.649267,Dimo's Pizza,41.945113,-87.654730,Pizza Place,4a3b2be5f964a52098a01fe3,8.8
625,Dearborn Park,41.866553,-87.628954,Flo & Santos,41.865398,-87.626123,Pizza Place,4bc7b41e92b376b08c4e503a,8.7
639,Dearborn Park,41.866553,-87.628954,Giordano's,41.864562,-87.624400,Pizza Place,5284eb42498e7b97a8388083,8.5
411,Buena Park,41.957810,-87.652833,Michael's Original Pizzeria & Tavern,41.956879,-87.651865,Pizza Place,4b0b3045f964a520552e23e3,8.4
581,Clearing West,41.780588,-87.773388,Vinces Pizza,41.777809,-87.776808,Pizza Place,4bef51ba5e4aa59319e758bb,8.1
568,Clearing East,41.780588,-87.773388,Vinces Pizza,41.777809,-87.776808,Pizza Place,4bef51ba5e4aa59319e758bb,8.1
1013,Forest Glen,41.978588,-87.751898,Mr V's Pizza,41.977261,-87.754474,Pizza Place,4bdb5e33a6fdef3bb51b1547,7.9
38,Andersonville,41.977139,-87.669273,Ranalli's,41.978054,-87.668855,Pizza Place,4a32e5bbf964a520bb9a1fe3,7.9
1351,Irving Park,41.952846,-87.729016,Lou Malnati's Pizzeria,41.953490,-87.732066,Pizza Place,56e9ebe5498e24bf3e783dc2,7.8
